In [2]:
import tensorflow as tf
import numpy as np

## tf.keras.layers.RNN
* Input shape:
    3-D tensor with shape `(batch_size, timesteps, features)`.

* Output shape:
    - If `return_state`: a list of tensors. The first tensor is
    the output. The remaining tensors are the last states,
    each with shape `(batch_size, state_size)`, where `state_size` could
    be a high dimension tensor shape.
    - If `return_sequences`: 3D tensor with shape
    `(batch_size, timesteps, output_size)`.

## tf.keras.layers.LSTMCell
    tf.keras.layers.LSTMCell(
        units,
        activation='tanh',
        recurrent_activation='sigmoid',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        recurrent_initializer='orthogonal',
        bias_initializer='zeros',
        unit_forget_bias=True,
        kernel_regularizer=None,
        recurrent_regularizer=None,
        bias_regularizer=None,
        kernel_constraint=None,
        recurrent_constraint=None,
        bias_constraint=None,
        dropout=0.0,
        recurrent_dropout=0.0,
        seed=None,
        **kwargs
    )

In [60]:
x = np.random.random((2,10,8))
rnn = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(3))
rnn(x)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 0.552522  , -0.38236696, -0.2097214 ],
       [ 0.57467663, -0.38092893, -0.23813564]], dtype=float32)>

In [68]:
rnn2 = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(3), return_sequences=False, return_state=True)
final_output,state_h,state_c = rnn2(x)